In [1]:
# from data_helper import readfile_to_dict
import numpy as np
import os
import cv2
from model_ML import create_model_pretrain, create_model_Conv3D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# import matplotlib.pyplot as plt

dim = (120,120)
n_sequence = 10
n_channels = 3
n_output = 5
batch_size = 1

X = np.empty((batch_size, n_sequence, *dim, n_channels)) # X : (n_samples, *dim, n_channels)
Y = np.empty((batch_size), dtype=int)


## Predict
# weights_path = 'BUPT-RGB-Crop-96-0.92-0.88.hdf5'
weights_path = 'BUPT-RGBdiff-crop-Conv3D-verytiny-dataset02-1600-0.88-0.77.hdf5'
#'BUPT-2d-equalsplit-RGBdif-72-0.98-0.90.hdf5'
# weights_path = 'KARD-aug-RGBdif-40-0.92-0.98.hdf5'
# model = create_model_pretrain(dim, n_sequence, n_channels, n_output, 'MobileNetV2')
model = create_model_Conv3D(dim, n_sequence, n_channels, n_output) 
model.load_weights(weights_path)


Using TensorFlow backend.


In [3]:
def calculateRGBdiff(sequence_img):
    'keep first frame as rgb data, other is use RGBdiff for temporal data'
    length = len(sequence_img)
    sh = sequence_img.shape
    new_sequence = np.zeros((sh[0],sh[1],sh[2],sh[3])) # (frame, w,h,3)

    # find RGBdiff frame 1 to last frame
    for i in range(length-1,0,-1): # count down
        new_sequence[i] = cv2.subtract(sequence_img[i],sequence_img[i-1])
    
    new_sequence[0] = sequence_img[0] # first frame as rgb data

    return new_sequence




def get_sampling_frame( len_frames):   
    '''
    Sampling n_sequence frame from video file
    Output: n_sequence index from sampling algorithm 
    '''     
    
    random_sample_range = 1
    # Randomly choose sample interval and start frame
    sample_interval = 4#np.random.randint(1, random_sample_range + 1)
    # print('sample_interval:',sample_interval)
    print(len_frames - sample_interval * n_sequence + 1, len_frames )
    start_i = np.random.randint(0, len_frames - sample_interval * n_sequence + 1)
   

    # sample_interval = len_frames//n_sequence
    # start_i = 0
    
    # Extract frames as tensors
    index_sampling = []
    end_i = sample_interval * n_sequence + start_i
    for i in range(start_i, end_i, sample_interval):
        if len(index_sampling) < n_sequence:
            index_sampling.append(i)
    
    return index_sampling


In [4]:
def run_one():
    

    # path_file = 'F:\\Master Project\\Dataset\\BasketBall-RGB\\'+action+'\\'+action+'00.mp4'
    
    cap = cv2.VideoCapture(path_file)
    length_file = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) # get length of frames,
    print(length_file)
    index_sampling = get_sampling_frame(length_file) # get index to sampling         
    for j, n_pic in enumerate(index_sampling):
        print(j, n_pic)

        cap.set(cv2.CAP_PROP_POS_FRAMES, n_pic)
        ret, frame = cap.read()
        # edge = cv2.Canny(frame,50,50)
        # frame = cv2.Laplacian(frame,cv2.CV_64F) 
        # frame = cv2.Sobel(frame,cv2.CV_64F,0,1,ksize=3)
        # # temp = edge/255.0
        # for c in range(3):
        #     frame[:,:,c] = edge

        new_image = cv2.resize(frame, dim)   
        new_image = new_image  
        # edge = cv2.Canny(new_image,100,200) 
        # temp = edge/255.0
        # for c in range(3):
        #     new_image[:,:,c] = temp

        X[0,j,:,:,:] = new_image
        
        # cv2.imshow('Frame',frame)


        # cv2.waitKey(500)

    cap.release()
    print(X.shape)

    X[0,] = sequence_augment(X[0,])/255.0  
    X[0,] = calculateRGBdiff(X[0,])

    for i in range(n_sequence):    
        cv2.imshow('Frame',X[0,i])
        cv2.waitKey(500)

    result = model.predict(X)
    print(np.around(result,3))
    print(np.max(result))



In [6]:
aug_gen = ImageDataGenerator(fill_mode='nearest')
def sequence_augment(sequence):
        name_list = ['rotate','width_shift','height_shift',
                    'brightness','flip_horizontal','width_zoom',
                    'height_zoom']
        # dictkey_list = ['theta','ty','tx',
        #             'brightness','flip_horizontal','zy',
        #             'zx']
        dictkey_list = ['zy','zx']#['ty','tx','theta','zy','zx']
        random_aug = 1 #np.random.randint(2, 5) # random 0-4 augmentation method
        pick_idx = np.random.choice(len(dictkey_list), random_aug, replace=False) #
        dict_input = {}
        for i in pick_idx:
            if dictkey_list[i] == 'theta':
                dict_input['theta'] = 5 #np.random.randint(-10, 10)

            elif dictkey_list[i] == 'ty': # width_shift
                dict_input['ty'] = 20#np.random.randint(-60, 60)

            elif dictkey_list[i] == 'tx': # height_shift
                dict_input['tx'] = 10#np.random.randint(-30, 30)

            elif dictkey_list[i] == 'brightness': 
                dict_input['brightness'] = np.random.uniform(0.15,1)

            elif dictkey_list[i] == 'flip_horizontal': 
                dict_input['flip_horizontal'] = True

            elif dictkey_list[i] == 'zy': # width_zoom
                dict_input['zy'] = 1.3#np.random.uniform(0.5,1.5)

            elif dictkey_list[i] == 'zx': # height_zoom
                dict_input['zx'] = 1.3#np.random.uniform(0.5,1.5)
        
        sh = sequence.shape
        new_sequence = np.zeros((sh[0],sh[1],sh[2],sh[3]))

        print(dict_input)
        for i in range(sh[0]):
            new_sequence[i] = aug_gen.apply_transform(sequence[i],dict_input)
        
        return new_sequence

In [18]:
sub_folder = 'peach'
action = 'walk'
base_path = 'F:\\Master Project\\'

# base_path = 'D:\\Peach\\'
# path_file = base_path+'Dataset\\sit_stand\\'+action+'\\'+action+'03_04.mp4'
# path_file = base_path+'Dataset\\KARD-split\\'+action+'\\'+action+'_s09_e03.mp4'
# path_file = base_path+'Dataset\\BUPT-dataset\\RGBdataset_crop\\'+sub_folder+'\\'+action+'\\'+action+'04_02_00.mp4'
path_file = base_path+'Dataset\\sit_stand_crop02\\'+sub_folder+'\\'+action+'\\'+action+'04_01_03.mp4'

run_one()

# run, sit, stand, walk, standup

218
179 218
0 116
1 120
2 124
3 128
4 132
5 136
6 140
7 144
8 148
9 152
(1, 10, 120, 120, 3)
{'zx': 1.3}
[[0.7 0.  0.  0.3 0. ]]
0.69972956
